# Heavy Commercial Vehicles' Mobility: Dataset of Trucks Anonymized Recorded Driving and Operation (DT-CARGO)

Authors: Balke, Georg; Adenaw, Lennart 

## Preparations
### Imports

In [ ]:
import zipfile

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import pandas as pd
import seaborn as sns

# from fclist import fclist

from IPython.display import Markdown as md

from smvis.gridfigure import GridFigure
from smvis.utils import setFont, setFontSize, genLineLegendHandle

### Constants

In [ ]:
# List of flags for plots and iterations
LOCATIONS = ['home_base', 'rest_area', 'service_area_fuel', 'industrial_area']

### Colors

In [ ]:
colors = ["#0065BD", "#000000", "#E37222", "#A2AD00",
          "#f7c420", "#b4ceb3", "#2f7ae0", "#1e5722", 
          "#f76c5e", "#808080" ]
colors_fleets = {i[0]:i[1] for i in zip(range(1,5), colors[:4])}
colors_locations = {i[0]:i[1] for i in zip([*LOCATIONS, 'other_area', 'driving'], colors[4:10])}
colors_locations_2 = {i[0].replace('_', ' '):i[1] for i in zip([*LOCATIONS, 'other_area', 'driving'], colors[4:10])}
# Set your custom color palette
sns.set_palette(sns.color_palette(colors))

# grid lines' alpha
alpha_major = 0.8
alpha_minor = 0.4

def swatches(colors, sep=' ', width=6):
    display(md(sep.join(
        f'<span style="font-family: monospace">{label}:{color}  <span style="color: {color}">{chr(9608)*width}</span></span><br>'
        for label, color in colors.items()
    )))    
    
swatches(colors_fleets)
swatches(colors_locations)

### Fonts and sizes

In [ ]:
textwidth = 159.2 / 25.4
h_169 = 9/16 * textwidth 
h_43 = 3/4 * textwidth

print(f'Standard plot sizes: {textwidth}" x {h_169:.2f}" (wide) & {textwidth}" x {h_43:.2f}" (narrow)')

# font modification if required
# arial_path = next(fclist(family="Arial", style="Regular")).file
# setFont(arial_path)

## Data Import

In [ ]:
# Vehicle and trip number to assess for speed profile
i_veh, i_trip = 1, 10

### Data Retrieval

In [ ]:
# trips of vehicles
df_trips_unfiltered = pd.read_csv('input/public/tracks.csv', index_col='track_id', parse_dates=['start_time','stop_time'])

# overview of vehicles and fleets
df_fleet = pd.read_csv('input/public/fleet.csv', index_col='vehicle_id')

# speed profile
zf = zipfile.ZipFile('input/public/speed.zip') 
df_speed = pd.read_csv(zf.open(f'speed/{i_veh}/{i_trip}.csv'))

### Preprocessing

In [ ]:
# filtering small trips for visualization
df_trips = df_trips_unfiltered.copy()
df_trips = df_trips.loc[df_trips_unfiltered.distance > 1000]

# distance (kilometers) and duration (hours) added to trips as columns for visualization
df_trips['distance_km'] = df_trips['distance'] / 1000
df_trips['duration'] = (df_trips.stop_time - df_trips.start_time).dt.total_seconds()
df_trips['duration_h'] = df_trips['duration'] / 3600

# fleet assigned to trips for grouping
df_trips['fleet_test_id'] = df_trips.join(df_fleet, on='vehicle_id', how='left')['fleet_test_id'].astype("category")

In [ ]:
# df_stops focuses on the DESTINATIONS of a trip. 
df_stops = df_trips.copy()

# four independent flags in data
# for evaluation purposes put in hierarchy. (specific matches before broad categories)
df_stops['service_area_fuel'] = df_trips.apply(lambda x: (not x['home_base']) and (x['service_area_fuel']), axis=1)
df_stops['rest_area']         = df_trips.apply(lambda x: (not x['home_base']) and (not x['service_area_fuel']) 
                                               and (x['rest_area']), axis=1)
df_stops['industrial_area']   = df_trips.apply(lambda x: (not x['home_base']) and (not x['service_area_fuel']) 
                                             and (not x['rest_area']) and (x['industrial_area']), axis=1)
df_stops['other_area']        = df_trips.apply(lambda x: (not x['home_base']) and (not x['service_area_fuel']) 
                                             and (not x['rest_area']) and (not x['industrial_area']), axis=1)

# boolean flags converted into string column
df_stops['location'] = df_stops[[*LOCATIONS, 'other_area']].idxmax(axis=1).str.replace('_', ' ')

# calculation of rest time at DESTINATION
df_stops = df_stops.assign(rest_time=(df_trips.groupby('vehicle_id').start_time.shift(-1) - df_trips.stop_time).dt.total_seconds())
df_stops = df_stops.assign(rest_time_h=df_stops['rest_time'] / 3600)

In [ ]:
# speed converted to kilometers per hour for visualization
df_speed["kmh"] = df_speed.speed*3.6

# relative time in seconds since beginning of trip
df_speed['rel_time'] = df_speed.epoch - df_speed.epoch.min()
df_speed.set_index('rel_time', inplace=True)

In [ ]:
# fleet occupation plots

df_rt_joined = pd.DataFrame(index=[1,2,3,4])
for flag in [*LOCATIONS, 'other_area']:
    rest_times = df_stops.loc[df_stops[flag]].groupby('fleet_test_id').sum().rest_time 
    if not len(df_rt_joined):
        df_rt_joined = pd.DataFrame(rest_times)
    else:
        df_rt_joined = df_rt_joined.join(rest_times, rsuffix=f'_{flag}')
        
# first column joined has no suffix --> give it one!
df_rt_joined.rename(columns={'rest_time':f'rest_time_{LOCATIONS[0]}'}, inplace=True)
# other columns --> remove "rest_time_" prefix
df_rt_joined.rename(columns={f'rest_time_{l}': l for l in [*LOCATIONS, 'other_area']}, inplace=True)

# add driving to dataframe that until now only has stops
df_driving = df_trips.groupby('fleet_test_id').duration.sum() 

df_rt_joined_plot = df_rt_joined.join(df_driving).rename(columns={'duration':'driving'})

In [ ]:
# What does the truck do?
# start_time and stop_time refer to DRIVING (track). thus, a break starts at "stop_time" and stops at "start time"
# columns have to be renamed to respect that
df_occupation = df_stops[['vehicle_id', 'fleet_test_id', 'start_time', 'stop_time']].copy()
df_occupation.rename(columns={'start_time':'stop_time', 'stop_time':'start_time'}, inplace=True)
df_occupation = df_occupation.assign(occupation=df_stops[[*LOCATIONS, 'other_area']].idxmax(axis=1))

df_driving_occupation = df_trips[['vehicle_id', 'fleet_test_id', 'start_time', 'stop_time']].copy()
df_driving_occupation['occupation'] ='driving'

df_occupation = df_occupation.append(df_driving_occupation.reset_index(drop=True))

df_occupation['start_time'] = pd.to_datetime(df_occupation['start_time'], utc=True)
df_occupation['stop_time'] = pd.to_datetime(df_occupation['stop_time'], utc=True)

df_occupation['duration'] = df_occupation.stop_time - df_occupation.start_time

#df_occupation.set_index(['vehicle_id','fleet_test_id','start_time'], inplace=True)
df_occupation['duration'] = df_occupation.duration.dt.total_seconds() / 3600
df_occupation.set_index('start_time', inplace=True)
df_occupation = df_occupation.tz_convert('Europe/Berlin')

## Resample occupation

resampled = df_occupation.groupby(['vehicle_id']).resample('1min').ffill()[['fleet_test_id', 'occupation', 'duration']]
resampled = resampled.reset_index()

resampled['dow'] = resampled.start_time.dt.dayofweek
resampled['hour'] = resampled.start_time.dt.hour
resampled = resampled.loc[resampled.dow < 6] # only week days
resampled = resampled.loc[resampled.duration < 24] # remove very long stays

truck_day = resampled.groupby(['occupation', 'hour']).occupation.count()
truck_day = truck_day.unstack(level=-1, fill_value=0)

## Analysis
### Metadata
 - global information about the dataset

In [ ]:
# Calculate meta data
total_distance = df_trips.distance.sum() 
total_time = df_trips.duration.sum()
trips = len(df_trips)
tours = len(df_trips[['vehicle_id','tour_id']].drop_duplicates())
fleet_size = len(df_fleet)

print(f'The total driven distance is: {total_distance/1000:,.2f}km')
print(f'The total operation time is: {total_time/3600:,.2f}h')
print(f'A fleet of {fleet_size} vehicles drove {tours} tours from {df_trips.start_time.min()} until {df_trips.start_time.max()}')
print('Median Vehicle driving distance km: ', df_trips.groupby('vehicle_id').distance.sum().median() / 1000)

### Bar Plot: Weekly Distance per fleet
- information about recorded period and length of recording per fleet. (marker: first and last track per fleet)

In [ ]:
fig, ax = plt.subplots(figsize=(textwidth, h_169))
indicator_level = 70000
# Preprocess
df_trips['start_time_7d'] = pd.to_datetime(df_trips.start_time,utc=True).dt.to_period("W-SUN")
distances_km = df_trips.groupby(['start_time_7d','fleet_test_id']).distance.sum().unstack() / 1000

# Plot
distances_km.plot.bar(stacked=True, xlabel='', ylabel='distance / km', ax=ax, color=colors_fleets)

# Adjust figure
x_year_ticks = [8.001,16.5,24.001]
x_week_ticks = np.arange(len(distances_km))

ax.set(
    ylim=(0,80000)
)


ax.yaxis.set_major_locator(ticker.MultipleLocator(10000))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(5000))
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda t,pos: f"{int(t):,}"))

ax.xaxis.set_minor_locator(ticker.FixedLocator(x_week_ticks))
ax.xaxis.set_minor_formatter(ticker.FuncFormatter(lambda t, pos: f"{distances_km.index[t].start_time.date().day:02d}-{distances_km.index[t].start_time.date().month:02d}"))

ax.xaxis.set_major_locator(ticker.FixedLocator(x_year_ticks))
ax.xaxis.set_major_formatter(ticker.FixedFormatter(["2021","|","2022"]))

ax.tick_params(axis="x", which="major", pad=4, labelrotation=0)
ax.tick_params(axis="x", which="minor", pad=20, labelrotation=90)
ax.grid(axis="y", which='major', alpha=alpha_major)
ax.grid(axis="y", which='minor', alpha=alpha_minor)

legend = ax.get_legend()

first_recordings = ((distances_km>0).reset_index(drop=True).cumsum() == 1).idxmax()
last_recordings = (distances_km.cumsum() == distances_km.cumsum().iloc[-1]).reset_index(drop=True).idxmax()

for i in range(len(first_recordings)):
     ax.plot([first_recordings.iloc[i], last_recordings.iloc[i]], [indicator_level + i*2000, indicator_level + i*2000],'--|', linewidth=1, markersize=7, color=colors_fleets[first_recordings.index[i]]) #  Zoom indicator

# Set font/font size
setFontSize([ax], 9)

plt.legend(ncol=2, title="fleet", loc=[0.72,0.6])

plt.savefig('figures/paper/fleet_calendar.svg', bbox_inches='tight')

### Line Plot: Sample Cycle
- microscopic view of recorded data

In [ ]:
# Options
idxmin_detail, idxmax_detail = 2400, 2700
indicator_level_kmh, max_speed = 75, 80

# Prepare figure
gf = GridFigure(1, 2, textwidth, h_169/2, wspace=0.1)

speed_overview_ax = gf.axes_list[0]
speed_detail_ax = gf.axes_list[1]

# Plot
df_speed.plot(y='kmh', ax=speed_overview_ax)
df_speed.plot(y='kmh', ax=speed_detail_ax)

speed_overview_ax_hdop = speed_overview_ax.twinx()
speed_detail_ax_hdop = speed_detail_ax.twinx()

df_speed.plot(y='hdop', ax=speed_overview_ax_hdop, color=colors_locations['driving'], linestyle='--')
df_speed.plot(y='hdop', ax=speed_detail_ax_hdop, color=colors_locations['driving'], linestyle='--')

for ax in gf.axes_list:
    ax.plot([df_speed.index[idxmin_detail],df_speed.index[idxmax_detail]], [indicator_level_kmh, indicator_level_kmh],
            "--v", linewidth=1, markersize=5) #  Zoom indicator

# Format figure
speed_overview_ax.set(xlim=(0,df_speed.index.max()))
speed_detail_ax.set(xlim=(df_speed.index[idxmin_detail],df_speed.index[idxmax_detail]))

speed_overview_ax.set(
    xlabel='time / s',
    ylabel='speed / km/h',
    title='full trip',
    ylim=(0,max_speed),
    xlim=(0,800),
)

speed_detail_ax.set(
    xlabel='time / s',
    title='acceleration ramp',
    ylim=(0,max_speed),
    yticklabels=[],
    xlim=(240,280),
)


speed_overview_ax_hdop.set(
    ylabel='',
    yticklabels=[],
    ylim=(0,1.5)
)

speed_detail_ax_hdop.set(
    ylabel='hdop / m',
    ylim=(0,1.5),
)


speed_overview_ax.xaxis.set_major_locator(ticker.MultipleLocator(100))
speed_overview_ax.xaxis.set_minor_locator(ticker.MultipleLocator(25))

speed_detail_ax.xaxis.set_major_locator(ticker.MultipleLocator(10))
speed_detail_ax.xaxis.set_minor_locator(ticker.MultipleLocator(2.5))

speed_overview_ax.yaxis.set_major_locator(ticker.MultipleLocator(20))
speed_overview_ax.yaxis.set_minor_locator(ticker.MultipleLocator(5))

speed_detail_ax.yaxis.set_major_locator(ticker.MultipleLocator(20))
speed_detail_ax.yaxis.set_minor_locator(ticker.MultipleLocator(5))

# Format figure
speed_overview_ax.grid(which='major', alpha=alpha_major)
speed_overview_ax.grid(which='minor', alpha=alpha_minor)
speed_detail_ax.grid(which='major', alpha=alpha_major)
speed_detail_ax.grid(which='minor', alpha=alpha_minor)

handles_1, labels_1 = speed_detail_ax_hdop.get_legend_handles_labels()
handles_2, labels_2 = speed_detail_ax.get_legend_handles_labels()

speed_overview_ax_hdop.get_legend().remove()
speed_detail_ax_hdop.get_legend().remove()

speed_overview_ax.get_legend().remove()
speed_detail_ax.get_legend().remove()

speed_detail_ax.legend(handles = handles_1+handles_2, labels=['hdop', 'speed'], ncol=1, loc=(0.025,.5))

for ax_i in gf.axes_list:
    for tick in ax_i.get_xticklabels():
        tick.set(rotation=90)#,ha='right')

# Set font sizes
setFontSize(gf.axes_list, 9)
        
plt.savefig('figures/paper/speed_profile.svg', bbox_inches='tight')

### Distance / Duration Histograms of Trips
 - colored by fleet

In [ ]:
gf_dis = GridFigure(1, 2, textwidth, h_169, wspace=0.25, constrained_layout=False)

sns.histplot(data=df_trips.loc[df_trips.duration_h <= 5], 
            x='duration_h',
            hue="fleet_test_id",
            palette=colors_fleets, 
             bins=40,
             element="step", fill=False,
            common_norm=False, ax=gf_dis.axes_list[0])

sns.histplot(data=df_trips.loc[df_trips.distance_km <= 400], 
            x="distance_km",
            hue="fleet_test_id",
            palette=colors_fleets,
             bins=40,
             element="step", fill=False,
            common_norm=False, ax=gf_dis.axes_list[1])


#gf_dis.axes_list[1].yaxis.tick_right()
#gf_dis.axes_list[1].set(ylabel='')

gf_dis.axes_list[0].xaxis.set_major_locator(ticker.MultipleLocator(1))
gf_dis.axes_list[1].xaxis.set_major_locator(ticker.MultipleLocator(100))

gf_dis.axes_list[0].xaxis.set_minor_locator(ticker.MultipleLocator(0.25))
gf_dis.axes_list[1].xaxis.set_minor_locator(ticker.MultipleLocator(25))

gf_dis.axes_list[0].yaxis.set_minor_locator(ticker.MultipleLocator(25))
gf_dis.axes_list[1].yaxis.set_minor_locator(ticker.MultipleLocator(100))

gf_dis.axes_list[0].get_legend().remove()
legend_1 = gf_dis.axes_list[1].get_legend()
legend_1.set(title='fleet')

gf_dis.axes_list[0].grid(which='major', alpha=alpha_major)
gf_dis.axes_list[1].grid(which='major', alpha=alpha_major)
gf_dis.axes_list[0].grid(which='minor', alpha=alpha_minor)
gf_dis.axes_list[1].grid(which='minor', alpha=alpha_minor)


gf_dis.axes_list[0].set(xlim=(0,5),
                        ylim=(0,800),
                        xlabel='duration / h',
                        ylabel='count',)
gf_dis.axes_list[1].set(xlim=(0,400),
                        ylim=(0,2500),
                        xlabel='distance / km',
                        ylabel=''
                       )

gf_dis.axes_list[1].yaxis.set_major_formatter(ticker.FuncFormatter(lambda t,pos: f"{int(t):,}"))

# Set font sizes
setFontSize(gf_dis.axes_list, 9)

plt.savefig('figures/paper/trip_duration.svg', bbox_inches='tight')

In [ ]:
print(f'Share of trips shorter than 5h: {len(df_trips.loc[df_trips.duration_h <= 5]) / len(df_trips)}')
print(f'Share of trips shorter than 400km: {len(df_trips.loc[df_trips.distance_km <= 400]) / len(df_trips)}')
df_trips.groupby('fleet_test_id').median()

### Rest Time Distribution
- overview and at certain stop locations.

In [ ]:
cut_time_1 = 2
cut_time_2 = 6
cut_time_max = 20
indicator_level_main = 2.5
indicator_level_sub_1 = 2.5
indicator_level_sub_2 = 0.4
indicator_color = '#202020'

gf_loc = GridFigure(1,1, textwidth, h_43/2, wspace=0.2, hspace=0.3, legend_mode='above', constrained_layout=False)
gf_loc_2 = GridFigure(1,2, textwidth, h_43/2, wspace=0.2, constrained_layout=False)

plot_data = df_stops.loc[(df_stops.rest_time <= 24*3600)]
plot_data_2h = df_stops.loc[(df_stops.rest_time <= cut_time_1*3600)]
plot_data_24h = df_stops.loc[(df_stops.rest_time > cut_time_2*3600) & (df_stops.rest_time <= 24*3600)]

for data, bw_adjust, ax in zip ([plot_data, plot_data_2h, plot_data_24h],
                               [.002, .2, .2],
                               [gf_loc.axes_list[0], *gf_loc_2.axes_list]):
    sns.kdeplot(data=data,
                x='rest_time_h',
                hue='location',
                bw_adjust=bw_adjust, # bw=bw_adjust*len(plot_data)**(-1./(1+4)) for each hue
                palette=colors_locations_2,
                cut=0, common_norm=False,
                ax=ax)

gf_loc.axes_list[0].plot([0, cut_time_1], [indicator_level_main]*2,
    "--v", linewidth=1, markersize=5, color=indicator_color) #  Zoom indicator
gf_loc.axes_list[0].plot([cut_time_2, cut_time_max], [indicator_level_main]*2,
    "--o", linewidth=1, markersize=5, color=indicator_color) #  Zoom indicator
gf_loc_2.axes_list[0].plot([0, cut_time_1], [indicator_level_sub_1]*2,
    "--v", linewidth=1, markersize=5, color=indicator_color) #  Zoom indicator
gf_loc_2.axes_list[1].plot([cut_time_2, cut_time_max], [indicator_level_sub_2]*2,
    "--o", linewidth=1, markersize=5, color=indicator_color) #  Zoom indicator

        
gf_loc.legend_ax.axis("off")

for ax, \
    major_x_grid, minor_x_grid, \
    major_y_grid, minor_y_grid, \
    xlim, ylim, \
    xlabel, ylabel \
    in zip([gf_loc.axes_list[0], *gf_loc_2.axes_list],
                                      [4,0.5,2],
                                      [.5,0.1,1],
                                      [0.5,0.5,0.1],
                                      [0.25,0.1,0.05],
                                      [(0,24), (0,cut_time_1), (cut_time_2,cut_time_max)],
                                      [(0,3), (0,3), (0,0.5)],
                                      ['rest time / h'] * 3,
                                      ['density'] * 2 + ['']):
    ax.grid(which='major', alpha=alpha_major)
    ax.grid(which='minor', alpha=alpha_minor)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(major_x_grid))
    ax.xaxis.set_minor_locator(ticker.MultipleLocator(minor_x_grid))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(major_y_grid))
    ax.yaxis.set_minor_locator(ticker.MultipleLocator(minor_y_grid))
    
    ax.set(xlim=xlim, ylim=ylim, xlabel=xlabel, ylabel=ylabel,)
    ax.get_legend().remove()
    
    setFontSize([ax], 9)
    

setFontSize(gf_loc.axes_list, 9)

handles = [genLineLegendHandle(c) for c in colors_locations_2.values()]
gf_loc.legend_ax.legend(handles=handles[:-1], labels=list(colors_locations_2.keys())[:-1], ncol=3, loc="center")

gf_loc.fig.savefig(f'figures/paper/rest_time_global.svg', bbox_inches='tight')
gf_loc_2.fig.savefig(f'figures/paper/rest_time_detail.svg', bbox_inches='tight')

# KDE computation: David W. Scott. Multivariate Density Estimation. Wiley, August 1992. "Following Scott’s rule [cite], the kernel density estimation uses a Gaussian kernel with bandwidth 0.2 · n^(−1/5) for n samples

### Data Quality
- gaps in recording in relation to recorded km

In [ ]:
gf = GridFigure(1, 3, textwidth * .75, h_169 *.75, wspace=0.8, constrained_layout=False)

df_track_gap = df_trips.groupby('vehicle_id').sum() 
df_track_gap['track_gap_km'] = df_track_gap.track_gap / 1000
df_track_gap['ratio'] = df_track_gap.track_gap / df_track_gap.distance

distances_ax = gf.axes_list[0]
gaps_ax = gf.axes_list[1]
ratios_ax = gf.axes_list[2]

sns.violinplot(data=df_track_gap.distance_km.values, ax=distances_ax, inner='stick', cut=0, color='#A0A0A0')
sns.violinplot(data=df_track_gap.track_gap_km.values, ax=gaps_ax, inner='stick', color='#A0A0A0')
sns.violinplot(data=df_track_gap.ratio.values, ax=ratios_ax, inner='stick', color='#A0A0A0')

distances_ax.set(
    ylabel="recorded distance / km",
    ylim=(0, 5*10**4)
)

gaps_ax.set(
    ylabel="track gap / km",
    ylim=(0, 2.5*10**3)
)

ratios_ax.set(
    ylabel="gap-to-distance ratio",
    ylim=(0, 0.7),
    yticks=np.arange(0,0.9, 0.2)
)
    
for ax in gf.axes_list:
    ax.grid("y", which='major', alpha=alpha_major)
    ax.xaxis.labelpad = 10
    ax.set(xticks=[])
    ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda t,pos: f"{t:,.1f}".replace('.0','')))

setFontSize(gf.axes_list, 9)
plt.savefig('figures/paper/data_quality.svg', bbox_inches='tight')

# KDE computation: David W. Scott. Multivariate Density Estimation. Wiley, August 1992. "The kernel density estimation uses a Gaussian kernel. The bandwith was chosen using Scott's rule."

### Fleet Occupation

- aggregated
- time of day

In [ ]:
# Prepare figure

gridfig = GridFigure(1,2,textwidth, h_169, wspace=0.35, hspace=.5, legend_mode="above", constrained_layout=False, ratios={'height':[1], 'width':[1, 2]})
ax = gridfig.axes_list[0]

# Plot
plot_data = (df_rt_joined_plot.transpose() / df_rt_joined_plot.transpose().sum()).transpose()
plot_data = plot_data[['driving','home_base','industrial_area','other_area','rest_area','service_area_fuel']]
plot_data.plot.bar(ax = ax, stacked=True, color=colors_locations, width=0.5)

# Adjust figure
ax.set(
    ylim=(0,1),
    yticks=np.arange(0,1.04,0.1),
    xlabel="fleet",
    ylabel="share of vehicle status by fleet"
)

ax.grid(axis="y", which='major', alpha=alpha_major)
ax.grid(axis="y", which='minor', alpha=alpha_minor)

# Create legend
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda t, pos: f"{round(t*100)}"+"%"))# + u'\u2006%'))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(.05))

handles,labels = ax.get_legend_handles_labels()
gridfig.legend_ax.axis("off")
gridfig.legend_ax.legend(handles = handles, labels=[l.replace("_"," ") for l in labels], ncol=3, loc="center")
ax.get_legend().remove()

# Adjust fonts
ax = gridfig.axes_list[1]

# Plot
plot_data = (truck_day / truck_day.sum()).transpose()
plot_data.plot.bar(ax=ax, stacked=True, color=colors_locations)

# Adjust figure
ax.set(
    ylim=(0,1),
    yticks=np.arange(0,1.04,0.1),
    xlabel="hour",
    ylabel="share of vehicle status by time"
)

# Create legend
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda t, pos: f"{round(t*100)}%"))
handles,labels = ax.get_legend_handles_labels()
gridfig.legend_ax.axis("off")
gridfig.legend_ax.legend(handles = handles, labels=[l.replace("_"," ") for l in labels], ncol=3, loc="center")
ax.get_legend().remove()

ax.yaxis.set_minor_locator(ticker.MultipleLocator(.05))

ax.xaxis.set_minor_locator(ticker.MultipleLocator(1))
ax.xaxis.set_major_locator(ticker.MultipleLocator(2))

ax.grid(axis="y", which='major', alpha=alpha_major)
ax.grid(axis="y", which='minor', alpha=alpha_minor)

for ax_i in gridfig.axes_list:
    for tick in ax_i.get_xticklabels():
        tick.set(rotation=0)#,ha='right')

# Adjust fonts
setFontSize(gridfig.axes_list+[gridfig.legend_ax],9)

# Save figure
plt.savefig('figures/paper/daily_status.svg', bbox_inches='tight')